In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, accuracy_score

# Importar librerías avanzadas
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [2]:
df = pd.read_csv("Amazon_Cell_Phones.csv")

In [3]:
df.head()

,ID,product_name,product_link,image_link,Price (Dollar),discount_percentage,price_before_discount,rating_out_of_5,number_of_ratings,brand,operating_system,RAM (GB),CPU,Storage (GB),screen_size (Inches),cellular_technology,model_name,cpu_model,available_colors
0,1,"Apple iPhone XR, 64GB, Black - Unlocked (Renewed)",https://www.amazon.com/Apple-iPhone-XR-Fully-U...,https://m.media-amazon.com/images/I/717KHGCJ6e...,168.39,5.93,179.0,4.4,64918.0,Apple,iOS 12,3.0,NaN,64.0,6.10,"LTE, CDMA, EV-DO, UMTS, HSPA, HSDPA, GSM, EDGE",iPhone XR,NaN,"Black, Blue, Coral, Red, White, Yellow"
1,2,"Apple iPhone 11, 64GB, Black - Unlocked (Renewed)",https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,https://m.media-amazon.com/images/I/612kg3rGyY...,230.00,0.00,NaN,4.2,52613.0,Apple,iOS 16,4.0,NaN,64.0,6.10,4G,iPhone 11,NaN,"Black, Green, Purple, Red, White, Yellow"
2,3,"Apple iPhone 12, 64GB, Black - Fully Unlocked ...",https://www.amazon.com/Apple-iPhone-12-64GB-Bl...,https://m.media-amazon.com/images/I/41bIlvE1rd...,287.00,0.00,NaN,4.1,25403.0,Apple,iOS 16,4.0,NaN,64.0,6.10,"5G, 4G LTE",iPhone 12,NaN,"(Product)Red, Black, Blue, Green, Purple, Red,..."
3,4,"OnePlus Nord CE 2,​ 5G Unlocked Android Smartp...",https://www.amazon.com/OnePlus-Unlocked-Androi...,https://m.media-amazon.com/images/I/61+Q6Rh3OQ...,329.99,0.00,NaN,4.3,25133.0,OnePlus,Android 12.0,8.0,NaN,128.0,6.43,5G,Nord CE 2,Others,NaN
4,5,"Apple iPhone XS, US Version, 64GB, Space Gray ...",https://www.amazon.com/Apple-iPhone-64GB-Space...,https://m.media-amazon.com/images/I/51I7GPdU6L...,182.00,0.00,NaN,4.2,21859.0,Apple,iOS 12,NaN,NaN,64.0,5.80,"GSM, CDMA, HSPA, EVDO, LTE, HSDPA",iPhone XS,NaN,"GOLD, Gold, SILVER, Silver, Space Gray"


In [4]:
df.shape

(3351, 19)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3351 entries, 0 to 3350
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID                     3351 non-null   int64  
 1   product_name           3351 non-null   object 
 2   product_link           3351 non-null   object 
 3   image_link             3351 non-null   object 
 4   Price (Dollar)         2347 non-null   float64
 5   discount_percentage    3351 non-null   float64
 6   price_before_discount  569 non-null    float64
 7   rating_out_of_5        3117 non-null   float64
 8   number_of_ratings      3117 non-null   float64
 9   brand                  3347 non-null   object 
 10  operating_system       2947 non-null   object 
 11  RAM (GB)               1894 non-null   float64
 12  CPU                    1222 non-null   object 
 13  Storage (GB)           3256 non-null   float64
 14  screen_size (Inches)   3163 non-null   float64
 15  cell

In [6]:
print(df['CPU'].value_counts(dropna=False))

CPU
NaN             2129
2 GHz            136
2.2 GHz           90
1.8 GHz           62
2.84 GHz          56
                ... 
2.5, 1.6 GHz       1
120 Hz             1
1.3, 2.4 GHz       1
1.8, 2.8 GHz       1
3.35E+3 MHz        1
Name: count, Length: 151, dtype: int64


In [7]:
#Quitamos las columnas con muchos datod faltantes
df.drop(['ID', 'product_name', 'product_link', 'image_link', 'available_colors', 'CPU', 'cpu_model', 'RAM (GB)', 'cellular_technology', 'operating_system', 'model_name'], axis = 1, inplace= True)

In [8]:
df.shape

(3351, 8)

In [9]:
# Calcular el precio antes del descuento si está ausente y el descuento no es nulo
df['price_before_discount'] = df.apply(
    lambda row: row['Price (Dollar)'] / (1 - row['discount_percentage'] / 100)
    if pd.isnull(row['price_before_discount']) and row['discount_percentage'] > 0
    else row['Price (Dollar)'] if pd.isnull(row['price_before_discount']) else row['price_before_discount'],
    axis=1
)

In [10]:
# Crear categorías para el precio (por ejemplo, 3 clases)
df['Price_Category'] = pd.cut(df['Price (Dollar)'], bins=[0, 200, 400, np.inf], labels=[0, 1, 2])


In [13]:
df.dropna(inplace=True)

In [12]:
df.describe()

,Price (Dollar),discount_percentage,price_before_discount,rating_out_of_5,number_of_ratings,Storage (GB),screen_size (Inches)
count,2347.000000,3351.000000,2347.000000,3117.000000,3117.000000,3256.000000,3163.000000
mean,278.421172,3.476780,298.864631,3.837632,401.905679,125.590943,6.058097
std,271.801032,10.419117,293.315441,0.708294,2026.345519,155.421699,5.503657
min,4.890000,0.000000,4.890000,1.000000,1.000000,0.000000,0.393701
25%,92.770000,0.000000,99.950000,3.600000,11.000000,16.000000,5.500000
50%,185.990000,0.000000,199.000000,4.000000,58.000000,64.000000,6.400000
75%,349.990000,0.000000,383.245000,4.200000,223.000000,128.000000,6.700000
max,1999.990000,93.000000,2332.710000,5.000000,64918.000000,1024.000000,255.905650


In [14]:
# Crear un objeto LabelEncoder
label_encoder = LabelEncoder()

# Ajustar y transformar los datos de la columna 'ciudades'
df['brand'] = label_encoder.fit_transform(df['brand'])

In [15]:
df.head()

,Price (Dollar),discount_percentage,price_before_discount,rating_out_of_5,number_of_ratings,brand,Storage (GB),screen_size (Inches),Price_Category
0,168.39,5.93,179.00,4.4,64918.0,15,64.0,6.10,0
1,230.00,0.00,230.00,4.2,52613.0,15,64.0,6.10,1
2,287.00,0.00,287.00,4.1,25403.0,15,64.0,6.10,1
3,329.99,0.00,329.99,4.3,25133.0,119,128.0,6.43,1
4,182.00,0.00,182.00,4.2,21859.0,15,64.0,5.80,0


In [16]:
df.shape

(1985, 9)

In [18]:
df.head()

,Price (Dollar),discount_percentage,price_before_discount,rating_out_of_5,number_of_ratings,brand,Storage (GB),screen_size (Inches),Price_Category
0,168.39,5.93,179.00,4.4,64918.0,15,64.0,6.10,0
1,230.00,0.00,230.00,4.2,52613.0,15,64.0,6.10,1
2,287.00,0.00,287.00,4.1,25403.0,15,64.0,6.10,1
3,329.99,0.00,329.99,4.3,25133.0,119,128.0,6.43,1
4,182.00,0.00,182.00,4.2,21859.0,15,64.0,5.80,0


In [21]:
df.to_csv('Prices_Clean.csv', index= False)